In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model

In [2]:
# Load a pre-trained MobileNetV2 model without top (classification) layers
base_model = MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights='imagenet')

# Build the U-Net-like text removal model on top of MobileNetV2
def build_text_removal_model(base_model):
    # Decoder part (U-Net)
    x = base_model.output
    x = Conv2DTranspose(128, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
    x = Concatenate()([x, base_model.get_layer('block_13_expand_relu').output])
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2DTranspose(64, (3, 3), activation='relu', strides=(2, 2), padding='same')(x)
    x = Concatenate()([x, base_model.get_layer('block_6_expand_relu').output])
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2DTranspose(1, (3, 3), activation='sigmoid', strides=(2, 2), padding='same')(x)  # Output mask

    # Create the model
    model = Model(inputs=base_model.input, outputs=x)
    return model

# Build the text removal model
model = build_text_removal_model(base_model)

# Load the input image
image = cv2.imread("C:\MajorProject\image_part_001 (22).png")

# Preprocess the image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (256, 256))  # Resize to match the model's input size
image = image / 255.0  # Normalize the pixel values to [0, 1]

# Predict the text mask
text_mask = model.predict(np.expand_dims(image, axis=0))[0]
text_mask = (text_mask * 255).astype(np.uint8)  # Convert to 8-bit mask

# Resize the text mask to match the input image size
text_mask = cv2.resize(text_mask, (image.shape[1], image.shape[0]))

# Inpaint text regions with surrounding content
inpainted_image = cv2.inpaint(image, text_mask, 7, cv2.INPAINT_NS)

# Save the inpainted image
cv2.imwrite('output.png', (inpainted_image * 255).astype(np.uint8))

1/1 [==============================] - 1s 748ms/step


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\photo\src\inpaint.cpp:768: error: (-210:Unsupported format or combination of formats) 8-bit, 16-bit unsigned or 32-bit float 1-channel and 8-bit 3-channel input/output images are supported in function 'icvInpaint'
